In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Aritra
[nltk_data]     Banerjee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Aritra
[nltk_data]     Banerjee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import re
import random
import string 
import cupy as np
import pandas as pd
import time as time
from tqdm import tqdm
from numpy.random import randn
import glob
#from collections import Counter
#from collections import defaultdict
import operator
from nltk.corpus import stopwords
#from numpy.random import randn

In [6]:
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 

DATA

In [7]:
# import pandas as pd 
# from sklearn.model_selection import train_test_split

# df=pd.read_csv('Tweets.csv')
# map_dic = {'neutral':0,'positive':1,'negative':2}
# df['label'] = df['airline_sentiment'].map(map_dic)
# trainingSet, testSet = train_test_split(df, test_size=0.2)
# train_data = pd.Series(trainingSet.label.values,index=trainingSet.text).to_dict()
# test_data = pd.Series(testSet.label.values,index=testSet.text).to_dict()

### Working Directory

In [8]:
import os
# os.chdir("/home/rohan/CMI/SEM_4/NLP/Assignment_3/Data/aclImdb_v1/aclImdb/")

In [9]:
# train =pd.read_csv("/content/drive/My Drive/Data/NLP/IMDB_csv/train.csv")
# test =pd.read_csv("/content/drive/My Drive/Data/NLP/IMDB_csv/test.csv")

# train.label = pd.Categorical(train.label)
# train['code'] = train.label.cat.codes

# test.label = pd.Categorical(test.label)
# test['code'] = test.label.cat.codes

# train_text=list(train.text)
# train_label= list(train.code)

# test_text=list(test.text)
# test_label= list(test.code)

### Training Data

In [10]:
train_text=[]
train_label=[]

for file in glob.glob("D:/Academics/CMI/MSC_4th_sem/NLP/My_work/Assignment_3/Data/aclImdb/train/pos/*"):
    f = open(file, encoding='utf-8').read()
    train_text.append(f)
    train_label.append(1)

for file in glob.glob("D:/Academics/CMI/MSC_4th_sem/NLP/My_work/Assignment_3/Data/aclImdb/train/neg/*"):
    f = open(file, encoding='utf-8').read()
    train_text.append(f)
    train_label.append(0)
    


### Test Data

In [11]:
test_text=[]
test_label=[] 

for file in glob.glob("D:/Academics/CMI/MSC_4th_sem/NLP/My_work/Assignment_3/Data/aclImdb/test/pos/*"):
    f = open(file, encoding='utf-8').read()
    test_text.append(f)
    test_label.append(1)

for file in glob.glob("D:/Academics/CMI/MSC_4th_sem/NLP/My_work/Assignment_3/Data/aclImdb/test/neg/*"):
    f = open(file, encoding='utf-8').read()
    test_text.append(f)
    test_label.append(0) 

Process

In [12]:
# # Create the vocabulary.
# vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
# #vocab.append("OTHERS")
# vocab_size = len(vocab)
# print('%d unique words found' % vocab_size) # 18 unique words found

### Preprocessing

In [13]:
def remove_digits(text): 
    result = re.sub(r'\d+', '', text) 
    return result 

# remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

# remove whitespace from text 
def remove_extra_space(text): 
    return  " ".join(text.split()) 

In [14]:
def part_preprocessing(text):
    text = text.lower()
    text = remove_digits(text)
    text = remove_punctuation(text)
    text = remove_extra_space(text)
    return text

In [15]:
for i in range(len(train_text)):
    train_text[i] = part_preprocessing(train_text[i])

In [16]:
for i in range(len(test_text)):
    test_text[i] = part_preprocessing(test_text[i])

In [17]:
train_text[0]

'bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt'

#### Convert Data in required format

In [18]:
train_data = {}

for a, b in zip(train_text, train_label):
    train_data[a]=  b

test_data = {}

for a, b in zip(test_text, test_label):
    test_data[a]=  b


#### No of Data points

In [19]:
print("No of Total Training Data points:",len(train_data.keys()))
print("No of Total Test Data points:",len(test_data.keys()))

No of Total Training Data points: 24903
No of Total Test Data points: 24800


In [20]:
stemmer = PorterStemmer() 
lemmatizer = WordNetLemmatizer() 

#### Find frequencies and document frequencies of the unique words

In [21]:
vocab_raw = []
count= {}
df = {}
for text in train_data.keys():
    
    for word in text.split(" "):
        word = stemmer.stem(word)
        word = lemmatizer.lemmatize(word, pos ='v')
        vocab_raw.append(word)
        try:
            count[word] += 1
        except:
            count[word] = 1
            try:
                df[word] = 1
            except:
                df[word] += 1

In [22]:
sum(count.values())

5778645

#### TF-IDF values

In [23]:
tf_idf_vals = []

for word in count.keys():
    
    N = len(train_data)
    tf = count[word] / sum(count.values())
    idf =  np.log(N/df[word])
    tf_idf = tf*idf
    tf_idf_vals.append((word,tf_idf))

In [24]:
#words = re.split(r'[;,?!\s]\s*', text)
#vocab_raw = [w for text in train_data.keys() for w in text.split(" "))]

In [25]:
# words =list(Counter(vocab_raw).keys())# equals to list(set(words))
# counts =list(Counter(vocab_raw).values())

In [26]:
print("Total number of words:",len(vocab_raw))
print("Total number of unique words:",len(tf_idf_vals))

Total number of words: 5778645
Total number of unique words: 89835


In [27]:
#vocab_list = list(zip(words, counts))
vocab_list = tf_idf_vals
vocab_list.sort(key=operator.itemgetter(1),reverse =True)

#### Filterd Vocabulary

In [28]:
vocab = [vocab_list[i][0] for i in range(len(vocab_list))]

#### Remove Stopwords

In [29]:
def remove_stopwords(tokens_list):
    all_stopwords = stopwords.words('english')
    all_stopwords.append('')
    all_stopwords.append('br')
    
    out = [word for word in tokens_list if not word in all_stopwords]
    return out

In [30]:
filtered_vocab = remove_stopwords(vocab)

In [31]:
#filtered_vocab = filtered_vocab
vocab_size = len(filtered_vocab)
print("Length of the vocabulary:",vocab_size)

Length of the vocabulary: 89725


In [32]:
filtered_vocab = filtered_vocab[:1999]
filtered_vocab.append("OTHERS")
vocab_size = len(filtered_vocab)
print("Length of the working vocabulary:",vocab_size)

Length of the working vocabulary: 2000


In [33]:
# Assign indices to each word.
word_to_idx = { w: i for i, w in enumerate(filtered_vocab) }
idx_to_word = { i: w for i, w in enumerate(filtered_vocab) }
#print(word_to_idx['thank']) # 16 (this may change)
print("Word corresponding to 5 th index:",idx_to_word[5]) # sad (this may change)

Word corresponding to 5 th index: one


### Create Input

In [34]:
import numpy as np

def encoded_inputs(text):
  '''
  Returns an array of one-hot vectors representing the words
  in the input text string.
  - text is a string
  - Each one-hot vector has shape (vocab_size, 1)
  '''
  inputs = []
  text = part_preprocessing(text)
  words = text.split(' ')
  words = remove_stopwords(words)

  for word in words:
    word = stemmer.stem(word)
    word = lemmatizer.lemmatize(word, pos ='v')
    encoded_word = np.zeros((vocab_size, 1))
    if word not in word_to_idx.keys():
      encoded_word[word_to_idx["OTHERS"]] = 1
    else:  
      encoded_word[word_to_idx[word]] = 1
    inputs.append(encoded_word)
  return inputs

### RNN

In [37]:
class Recurrent_NN:

  def __init__(self, input_size,  hidden_size ,output_size, learning_rate= 0.001):
        
    # Initialize Weights and biases

    ## Weights corresponding to input and hidden layer
    self.Wxh = randn(hidden_size, input_size) / 1000

    ## Weights corresponding to two hidden layers
    self.Whh = randn(hidden_size, hidden_size) / 1000

    ## Weights corresponding to ouput and hidden layer
    self.Why = randn(output_size, hidden_size) / 1000

    ## Biases corresponding to hidden layer
    self.bh = np.zeros((hidden_size, 1))

    ## Biases corresponding to output layer
    self.by = np.zeros((output_size, 1))

    ## Learning rate
    self.learning_rate = learning_rate
  
  ## Activation Functions

  ## Tanh activation

  def tanh(self,x):
    return np.tanh(x)
  
  ## Softmax Activation

  def softmax(self,x):
    return np.exp(x) / sum(np.exp(x))

  ## Update parameter using gradient descent algorithm

  def update_param(self, param_grad_pair):
     x =param_grad_pair[0]
     d_x = param_grad_pair[1]
     x -= self.learning_rate * d_x
     return x

  ## To overcome exploding gradient problem in backprop

  def not_explode_grad(self,x):
     x = np.clip(x, -1, 1)
     return x
    
  ## Compute loss in the forward prop

  def calculate_loss(self,probs,target):
     loss = - np.log(probs[target])
     return loss

  ## Forward Propagation

  def forward_prop(self, inputs,target):

    ## Inputs and targets
    self.inputs = inputs
    self.target = int(target)

    ## Store h values in different time steps (Memory of RNN)
    self.h_values = {}

    ## Initialize the hidden node values 
    h = np.zeros((self.Whh.shape[0], 1))
    self.h_values[0] = h
    
    for i, x in enumerate(inputs):
      
      ## Previous hidden layer values is being used here
      Z = self.Wxh @ x + self.Whh @ h + self.bh

      ## Tanh activation on hidden layer
      h = self.tanh(Z)

      ## Store the current h for next time step
      self.h_values[i + 1] = h

    ## Compute output in the final time step
    y = self.Why @ h + self.by
    
    ## Softmax for probabilities
    probs = self.softmax(y)
    self.probs = probs

    ## Calculate Loss
    loss = self.calculate_loss(probs,target)
    
    return(y, h, probs, loss)

  
  ## Backpropagation    

  def BPTT(self):
    
    ## Gradient of loss w.r.t y
    d_y = self.probs
    d_y[self.target] -= 1
    
    # Initialize the gradients of loss w.r.t the paramters
    d_Whh = np.zeros(self.Whh.shape)
    d_Wxh = np.zeros(self.Wxh.shape)
    d_bh = np.zeros(self.bh.shape)

    ## No of inputs for a input data
    N = len(self.inputs)


    # Following gradient depends only on ouput and last time step hidden values
    d_Why = d_y @ self.h_values[N].T
    d_by = d_y

    # Gradient of loss w.r.t last time step h values
    d_h = self.Why.T @ d_y

    ## Backpropagate through time.
    for t in reversed(range(N-1,-1,-1)):

      # Derivative of tanh(x) w.r.t x is (1- tanh(x))^2
      ## Need the following value in computation of gradients
      temp = ((1 - self.h_values[t + 1] ** 2) * d_h)

      # Gradient of loss w.r.t bh
      d_bh += temp

      # Gradient of loss w.r.t Whh
      d_Whh += temp @ self.h_values[t].T

      # Gradient of loss w.r.t Wxh
      d_Wxh += temp @ self.inputs[t].T

      # Gradient of loss w.r.t h
      d_h = self.Whh @ temp

    ## Get rid of exploding gradients.
  
    d_Wxh, d_Whh, d_Why, d_bh, d_by = list(map(self.not_explode_grad,[d_Wxh, d_Whh, d_Why, d_bh, d_by]))

    ## Update weights and biases using gradient descent.

    param_grad_pair = [(self.Whh,d_Whh),(self.Wxh,d_Wxh),(self.Why,d_Why),(self.bh,d_bh),(self.by,d_by)]
    
    self.Whh,self.Wxh,self.Why,self.bh,self.by = list(map(self.update_param,param_grad_pair))
    

### Function for training and testing

In [45]:
def compute_loss_accuracy(data, BPTT=True):

    total_cost = 0
    correct_pred= 0

    items = list(data.items())
    ## No of total data points
    N = len(items)
    ## Shuffle the data
    random.shuffle(items)

    

    for x, y in items:

        inputs = encoded_inputs(x)
        true_label = int(y)
        
        print(inputs)

#         out, h, probs, loss = model.forward_prop(inputs,true_label)
#         total_cost += loss
#         correct_pred += int(np.argmax(probs) == true_label)

#         ## For test data we don't do backpropagation
#         if BPTT:
#           model.BPTT()
    
#     avg_loss = total_cost / N
#     accuracy = correct_pred / N

#     return(avg_loss, accuracy)

In [40]:
print("Length of Training Data:",len(train_data.keys()))
print("Length of Test Data:",len(test_data.keys()))

Length of Training Data: 24903
Length of Test Data: 24800


### Training and Testing

In [44]:
input_size = vocab_size
hidden_size = 20
output_size = 2
learning_rate = 0.001
model = Recurrent_NN(input_size,hidden_size, output_size, learning_rate)

# Training loop
for epoch in tqdm(range(10)):

    start = time.time()

    training_loss, training_accuracy = compute_loss_accuracy(train_data)  
    print('Epoch {}'.format(epoch + 1)) 
    print('Training Loss: {} and Training Accuracy: {}'.format(training_loss, training_accuracy))

    test_loss, test_accuracy = compute_loss_accuracy(test_data, BPTT=False)
    print('Test Loss: {} and Test Accuracy: {}'.format(test_loss, test_accuracy))

    end = time.time()

    print("Time taken to complete epoch {}: {}".format(epoch + 1, end - start))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch 1
Training Loss: [0.6933359] and Training Accuracy: 0.49411717463759386


 10%|████████                                                                        | 1/10 [14:41<2:12:15, 881.71s/it]

Test Loss: [0.6931456] and Test Accuracy: 0.5016129032258064
Time taken to complete epoch 1: 881.712171792984
Epoch 2
Training Loss: [0.69315242] and Training Accuracy: 0.501586154278601


 20%|████████████████                                                                | 2/10 [29:27<1:57:43, 882.88s/it]

...

Test Loss: [0.6932791] and Test Accuracy: 0.5016129032258064
Time taken to complete epoch 8: 884.5656213760376
Epoch 9
Training Loss: [0.69332807] and Training Accuracy: 0.4978516644580974


 90%|████████████████████████████████████████████████████████████████████████        | 9/10 [2:12:41<14:44, 884.73s/it]

Test Loss: [0.69314237] and Test Accuracy: 0.5016129032258064
Time taken to complete epoch 9: 885.289452791214
Epoch 10
Training Loss: [0.69321416] and Training Accuracy: 0.5032325422639843


100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [2:27:25<00:00, 884.57s/it]

Test Loss: [0.69316694] and Test Accuracy: 0.49838709677419357
Time taken to complete epoch 10: 884.172554731369


# Rough

In [35]:
def compute_loss_accuracy(data, BPTT=True):

    total_cost = 0
    correct_pred= 0

    items = list(data.items())
    ## No of total data points
    N = len(items)
    ## Shuffle the data
    random.shuffle(items)

    

    for x, y in items:

        inputs = encoded_inputs(x)
        true_label = int(y)
        
        print(np.array(inputs).shape)
        break

#         out, h, probs, loss = model.forward_prop(inputs,true_label)
#         total_cost += loss
#         correct_pred += int(np.argmax(probs) == true_label)

#         ## For test data we don't do backpropagation
#         if BPTT:
#           model.BPTT()
    
#     avg_loss = total_cost / N
#     accuracy = correct_pred / N

#     return(avg_loss, accuracy)
    return(inputs,0)

In [38]:
input_size = vocab_size
hidden_size = 20
output_size = 2
learning_rate = 0.001
model = Recurrent_NN(input_size,hidden_size, output_size, learning_rate)

# Training loop
for epoch in tqdm(range(10)):

    start = time.time()

    training_loss, training_accuracy = compute_loss_accuracy(train_data)  
    print('Epoch {}'.format(epoch + 1))
    break
#     print('Training Loss: {} and Training Accuracy: {}'.format(training_loss, training_accuracy))

    test_loss, test_accuracy = compute_loss_accuracy(test_data, BPTT=False)
#     print('Test Loss: {} and Test Accuracy: {}'.format(test_loss, test_accuracy))

    end = time.time()

#     print("Time taken to complete epoch {}: {}".format(epoch + 1, end - start))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

(78, 2000, 1)
Epoch 1


In [43]:
training_loss

[array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0